In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Conv1D, GlobalAvgPool2D, GlobalAveragePooling1D, \
    Dropout, Dense, Activation, Concatenate, Multiply, MaxPool2D, Add,  \
    LSTM, Bidirectional, Conv2D, AveragePooling2D, BatchNormalization, Flatten, GlobalAveragePooling2D, \
    GlobalMaxPooling2D, Reshape, Permute, multiply, Lambda, add, subtract, MaxPooling2D, GRU, ReLU, MaxPooling1D 
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score, roc_curve
from keras.models import Model, load_model
from keras import backend as K
from sklearn.model_selection import KFold
from keras.layers import Embedding
import matplotlib as mpl
import matplotlib.pyplot as plt
from keras.regularizers import l1, l2
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from Bio import SeqIO
mpl.use('TkAgg')
import warnings
warnings.filterwarnings("ignore")
import math
from tensorflow.keras import layers
from tensorflow import keras
from keras import initializers

2024-06-12 12:13:35.559641: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 12:13:35.595087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 12:13:36.172224: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # 不全部占满显存，按需分配当allow_growth设置为True时，分配器将不会指定所有的GPU内存，而是根据需求增长
session = tf.compat.v1.Session(config=config)

2024-06-12 12:13:40.865334: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8956 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


In [3]:
def read_fasta(file_path):
    '''File_path: Path to the fasta file
       Returns: List of sequence
    '''
    one=list(SeqIO.parse(file_path,'fasta'))
    return one

In [4]:
def protein_to_Kmer(seqs,K):
    numeric_sequences = []
    base_to_index = {'A': 0, 'C': 1, 'D': 2, 'E': 3,
                     'F': 4, 'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9, 'M': 10,
                     'N': 11, 'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16, 
                     'V': 17, 'W': 18, 'Y': 19,'X': 20
                     }
    for dna_sequence in seqs:
        numeric_sequence = []
        for base in dna_sequence:
            numeric_sequence.append(base_to_index[base])

        kmer_sequence = []
        for i in range(len(numeric_sequence) - (K - 1)):
            kmer = numeric_sequence[i:i + K]
            kmer_marge = [int("".join(map(str, kmer)))]
            kmer_sequence.append(kmer_marge)

        numeric_sequences.append(kmer_sequence)
    return numeric_sequences

In [5]:
# Read the training set
    
train_pos_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_pos_15_train.txt'))
train_neg_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_neg_15_train.txt'))

train_seqs = np.concatenate( (train_pos_seqs, train_neg_seqs), axis=0 )

train = np.array(protein_to_Kmer(train_seqs,1)).astype(np.float32)

train_label = np.array( [1] * 2866 + [0] * 2866 ).astype( np.float32 )
train_label = to_categorical( train_label, num_classes=2 )

# Read the testing set
test_pos_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_pos_15_test.txt'))
test_neg_seqs = np.array(read_fasta('./data_Kbhb/train_15/Kbhb_neg_15_test.txt'))

test_seqs = np.concatenate((test_pos_seqs, test_neg_seqs), axis=0)

test = np.array(protein_to_Kmer(test_seqs,1)).astype(np.float32)

test_label = np.array([1] * 318 + [0] * 318).astype(np.float32)
test_label = to_categorical(test_label, num_classes=2)

In [6]:
# Performance evaluation
def show_performance(y_true, y_pred):

    TP, FP, FN, TN = 0, 0, 0, 0

    for i in range(len(y_true)):
        if y_true[i] == 1:
            if y_pred[i] > 0.5:
                TP += 1
            else:
                FN += 1
        if y_true[i] == 0:
            if y_pred[i] > 0.5:
                FP += 1
            else:
                TN += 1


    Sn = TP / (TP + FN + 1e-06)

    Sp = TN / (FP + TN + 1e-06)

    Acc = (TP + TN) / len(y_true)

    MCC = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) + 1e-06)

    return Sn, Sp, Acc, MCC

def performance_mean(performance):
    print('Sn = %.4f ± %.4f' % (np.mean(performance[:, 0]), np.std(performance[:, 0])))
    print('Sp = %.4f ± %.4f' % (np.mean(performance[:, 1]), np.std(performance[:, 1])))
    print('Acc = %.4f ± %.4f' % (np.mean(performance[:, 2]), np.std(performance[:, 2])))
    print('Mcc = %.4f ± %.4f' % (np.mean(performance[:, 3]), np.std(performance[:, 3])))
    print('Auc = %.4f ± %.4f' % (np.mean(performance[:, 4]), np.std(performance[:, 4])))

In [7]:
# Building the model
def build_model(windows=15,embed_dim=64, weight_decay=1e-4,num_heads=8,ff_dim=64):

    print('*' * 30 + 'GRU' + '*' * 30)
    input_1 = Input(shape=(windows,))
    # Word embedding coding
    embedding = Embedding(21, embed_dim, input_length=15)
    x_1 = embedding(input_1)
    print(x_1.shape)
    #transformer = TransformerBlock(embed_dim, num_heads, ff_dim)(x_1)
    #print(transformer.shape)

    x_1 = GRU(32,return_sequences=True,activation = 'relu')(x_1)
    x_1 = Dropout(0.5)(x_1)
    x_1 = GRU(16,return_sequences=True,activation = 'relu')(x_1)
    x_1 = Dropout(0.5)(x_1)
    print(x_1.shape)
    
    x = Flatten()(x_1)
    x = Dropout(0.5)(x)
    x = Dense(units=2, activation='softmax')(x)


    inputs = [input_1]
    outputs = [x]

    model = Model(inputs=inputs, outputs=outputs, name="Kbhb")

    optimizer = Adam(learning_rate=1e-3, epsilon=1e-7)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [8]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('./models_duibi_GRU/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./models_duibi_GRU/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./models_duibi_GRU/model_test.h5')

del model

model = load_model('./models_duibi_GRU/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************GRU******************************
(None, 15, 64)


2024-06-03 08:56:45.618018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8029 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 16)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-03 08:56:50.129755: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-03 08:56:50.233965: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc22c007150 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-03 08:56:50.234021: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-03 08:56:50.241966: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-03 08:56:50.440067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
2024-06-03 08:56:50.483250: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. 

  1/162 [..............................] - ETA: 13:44 - loss: 0.6930 - accuracy: 0.4688

...

162/162 [==============================] - 12s 74ms/step - loss: 0.5524 - accuracy: 0.7160 - val_loss: 0.5617 - val_accuracy: 0.7334
Epoch 19/200
162/162 [==============================] - 12s 73ms/step - loss: 0.5531 - accuracy: 0.7216 - val_loss: 0.5591 - val_accuracy: 0.7265
Epoch 20/200
162/162 [==============================] - 12s 74ms/step - loss: 0.5421 - accuracy: 0.7206 - val_loss: 0.5515 - val_accuracy: 0.7387
Epoch 21/200
162/162 [==============================] - 12s 73ms/step - loss: 0.5440 - accuracy: 0.7204 - val_loss: 0.5502 - val_accuracy: 0.7178
Epoch 22/200
162/162 [==============================] - 12s 73ms/step - loss: 0.5381 - accuracy: 0.7361 - val_loss: 0.5327 - val_accuracy: 0.7369
Epoch 23/200
162/162 [==============================] - 12s 74ms/step - loss: 0.5382 - accuracy: 0.7321 - val_loss: 0.5700 - val_accuracy: 0.7300
Epoch 24/200
162/162 [==============================] - 12s 74ms/step - loss: 0.5278 - accuracy: 0.7301 - val_loss: 0.5400 - val_accuracy

2024-06-03 09:03:36.940238: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 12s 74ms/step - loss: 0.4979 - accuracy: 0.7617 - val_loss: 0.5270 - val_accuracy: 0.7526
Epoch 35/200
162/162 [==============================] - 12s 74ms/step - loss: 0.4882 - accuracy: 0.7561 - val_loss: 0.5324 - val_accuracy: 0.7596
Epoch 36/200
162/162 [==============================] - 12s 73ms/step - loss: 0.4869 - accuracy: 0.7625 - val_loss: 0.5319 - val_accuracy: 0.7509
Epoch 37/200
162/162 [==============================] - 12s 73ms/step - loss: 0.4754 - accuracy: 0.7712 - val_loss: 0.5635 - val_accuracy: 0.7404
Epoch 38/200
162/162 [==============================] - 12s 73ms/step - loss: 0.4846 - accuracy: 0.7683 - val_loss: 0.5237 - val_accuracy: 0.7439
Epoch 39/200
162/162 [==============================] - 12s 74ms/step - loss: 0.4861 - accuracy: 0.7656 - val_loss: 0.5476 - val_accuracy: 0.7404
Epoch 40/200
162/162 [==============================] - 12s 74ms/step - loss: 0.4743 - accuracy: 0.7769 - val_loss: 0.5615 - val_accuracy

2024-06-03 09:36:37.271741: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 13s 74ms/step - loss: 0.1319 - accuracy: 0.9536 - val_loss: 3.4957 - val_accuracy: 0.6761
Epoch 3/200
180/180 [==============================] - 14s 76ms/step - loss: 0.1871 - accuracy: 0.9363 - val_loss: 2.0010 - val_accuracy: 0.6761
Epoch 4/200
180/180 [==============================] - 13s 75ms/step - loss: 0.1208 - accuracy: 0.9522 - val_loss: 2.3226 - val_accuracy: 0.6950
Epoch 5/200
180/180 [==============================] - 13s 74ms/step - loss: 0.1182 - accuracy: 0.9548 - val_loss: 2.5576 - val_accuracy: 0.6887
Epoch 6/200
180/180 [==============================] - 13s 73ms/step - loss: 0.1036 - accuracy: 0.9621 - val_loss: 2.7949 - val_accuracy: 0.6777
Epoch 7/200
180/180 [==============================] - 13s 74ms/step - loss: 0.1323 - accuracy: 0.9525 - val_loss: 2.1955 - val_accuracy: 0.6745
Epoch 8/200
180/180 [==============================] - 13s 73ms/step - loss: 0.1102 - accuracy: 0.9583 - val_loss: 2.5957 - val_accuracy: 0.69

In [8]:
np.random.seed(0)
tf.random.set_seed(1)  # for reproducibility
# reading model

model = build_model()


BATCH_SIZE = 32
EPOCHS = 200

# # Cross-validation
n = 10
k_fold = KFold(n_splits=n, shuffle=True, random_state=42)

all_performance = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)
for fold_count, (train_index, val_index) in enumerate(k_fold.split(train)):
    print('*' * 30 + ' the ' + str(fold_count + 1) + ' fold ' + '*' * 30)
    trains, val = train[train_index], train[val_index]
    trains_label, val_label = train_label[train_index], train_label[val_index]
    model.fit(x=trains, y=trains_label, validation_data=(val, val_label), epochs=EPOCHS,
                batch_size=BATCH_SIZE, shuffle=True,
                callbacks=[EarlyStopping(monitor='val_loss', patience=13, mode='auto')],
                verbose=1)
     # 保存模型

    model.save('./models_duibi_GRU/model_fold' + str(fold_count+1) + '.h5')

    del model

    model = load_model('./models_duibi_GRU/model_fold' + str(fold_count+1) + '.h5')

    val_pred = model.predict(val, verbose=1)

    # Sn, Sp, Acc, MCC, AUC
    Sn, Sp, Acc, MCC = show_performance(val_label[:, 1], val_pred[:, 1])
    AUC = roc_auc_score(val_label[:, 1], val_pred[:, 1])
    print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

    performance = [Sn, Sp, Acc, MCC, AUC]
    all_performance.append(performance)
    
all_performance = np.array(all_performance)
print('10 fold result:', all_performance)
performance_mean = performance_mean(all_performance)

model.fit(x=train, y=train_label, validation_data=(test, test_label), epochs=EPOCHS,
                      batch_size=BATCH_SIZE, shuffle=True,
                      callbacks=[EarlyStopping(monitor='val_loss', patience=20, mode='auto')],
                      verbose=1)
model.save('./models_duibi_GRU/model_test.h5')

del model

model = load_model('./models_duibi_GRU/model_test.h5')

test_score = model.predict(test)


# Sn, Sp, Acc, MCC, AUC
Sn, Sp, Acc, MCC = show_performance(test_label[:,1], test_score[:,1])
AUC = roc_auc_score(test_label[:,1], test_score[:,1])

print('-----------------------------------------------test---------------------------------------')
print('Sn = %f, Sp = %f, Acc = %f, MCC = %f, AUC = %f' % (Sn, Sp, Acc, MCC, AUC))

******************************GRU******************************
(None, 15, 64)


2024-06-12 12:14:04.813414: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8956 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


(None, 15, 16)
****************************** the 1 fold ******************************
Epoch 1/200


2024-06-12 12:14:08.453303: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-12 12:14:08.516030: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5638d81536a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-12 12:14:08.516080: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-12 12:14:08.526404: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-12 12:14:08.697895: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8800
2024-06-12 12:14:08.743091: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. 

  1/162 [..............................] - ETA: 11:03 - loss: 0.6938 - accuracy: 0.4375

...

162/162 [==============================] - 12s 72ms/step - loss: 0.5558 - accuracy: 0.7061 - val_loss: 0.5701 - val_accuracy: 0.7230
Epoch 19/200
162/162 [==============================] - 11s 70ms/step - loss: 0.5565 - accuracy: 0.7127 - val_loss: 0.5740 - val_accuracy: 0.7300
Epoch 20/200
162/162 [==============================] - 11s 69ms/step - loss: 0.5473 - accuracy: 0.7231 - val_loss: 0.5643 - val_accuracy: 0.7213
Epoch 21/200
162/162 [==============================] - 11s 69ms/step - loss: 0.5418 - accuracy: 0.7284 - val_loss: 0.5638 - val_accuracy: 0.7352
Epoch 22/200
162/162 [==============================] - 11s 70ms/step - loss: 0.5376 - accuracy: 0.7309 - val_loss: 0.5412 - val_accuracy: 0.7265
Epoch 23/200
162/162 [==============================] - 11s 70ms/step - loss: 0.5314 - accuracy: 0.7311 - val_loss: 0.5744 - val_accuracy: 0.7300
Epoch 24/200
162/162 [==============================] - 11s 70ms/step - loss: 0.5231 - accuracy: 0.7404 - val_loss: 0.5505 - val_accuracy

2024-06-12 12:20:37.051413: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



162/162 [==============================] - 11s 69ms/step - loss: 0.4918 - accuracy: 0.7623 - val_loss: 0.5207 - val_accuracy: 0.7753
Epoch 35/200
162/162 [==============================] - 11s 69ms/step - loss: 0.4829 - accuracy: 0.7703 - val_loss: 0.5278 - val_accuracy: 0.7613
Epoch 36/200
162/162 [==============================] - 11s 69ms/step - loss: 0.4875 - accuracy: 0.7604 - val_loss: 0.5387 - val_accuracy: 0.7544
Epoch 37/200
162/162 [==============================] - 11s 69ms/step - loss: 0.4748 - accuracy: 0.7716 - val_loss: 0.5573 - val_accuracy: 0.7456
Epoch 38/200
162/162 [==============================] - 11s 69ms/step - loss: 0.4841 - accuracy: 0.7691 - val_loss: 0.5232 - val_accuracy: 0.7491
Epoch 39/200
162/162 [==============================] - 11s 69ms/step - loss: 0.4858 - accuracy: 0.7675 - val_loss: 0.5512 - val_accuracy: 0.7474
Epoch 40/200
162/162 [==============================] - 11s 70ms/step - loss: 0.4723 - accuracy: 0.7776 - val_loss: 0.5366 - val_accuracy

2024-06-12 12:49:46.217124: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 9.1.108, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.



180/180 [==============================] - 12s 66ms/step - loss: 0.1916 - accuracy: 0.9196 - val_loss: 1.4906 - val_accuracy: 0.7060
Epoch 3/200
180/180 [==============================] - 12s 67ms/step - loss: 0.1932 - accuracy: 0.9245 - val_loss: 1.2811 - val_accuracy: 0.7217
Epoch 4/200
180/180 [==============================] - 12s 67ms/step - loss: 0.1702 - accuracy: 0.9257 - val_loss: 1.6975 - val_accuracy: 0.7075
Epoch 5/200
180/180 [==============================] - 12s 66ms/step - loss: 0.1770 - accuracy: 0.9243 - val_loss: 1.7613 - val_accuracy: 0.7107
Epoch 6/200
180/180 [==============================] - 12s 65ms/step - loss: 0.1881 - accuracy: 0.9213 - val_loss: 1.6636 - val_accuracy: 0.7138
Epoch 7/200
180/180 [==============================] - 12s 66ms/step - loss: 0.1931 - accuracy: 0.9189 - val_loss: 1.5326 - val_accuracy: 0.7264
Epoch 8/200
180/180 [==============================] - 12s 66ms/step - loss: 0.1968 - accuracy: 0.9227 - val_loss: 1.2555 - val_accuracy: 0.72

In [9]:
df = pd.DataFrame({
    'true_label': test_label[:,1],
    'predicted_probability': test_score[:, 1]  # 提取正类的预测概率
})
df.to_csv('./models_duibi_GRU/predictions_with_probabilities_gru.csv', index=False)